Staging Layer

In [27]:
import pandas as pd
employee = pd.read_csv('../Source/employee.csv')
course = pd.read_csv('../Source/courses.csv')
employee_course = pd.read_csv('../Source/employee_course.csv')

In [28]:
employee_df = pd.DataFrame(employee)
course_df = pd.DataFrame(course)
employee_course_df = pd.DataFrame(employee_course)

In [29]:
print(employee.info())
print(course.info())
print(employee_course_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                500 non-null    int64 
 1   empID             500 non-null    object
 2   name              500 non-null    object
 3   designation       500 non-null    object
 4   password          500 non-null    object
 5   email             500 non-null    object
 6   mobile            500 non-null    int64 
 7   performance_rate  500 non-null    int64 
dtypes: int64(3), object(5)
memory usage: 31.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                50 non-null     int64 
 1   courseID          50 non-null     object
 2   name              50 non-null     object
 3   duration          50 non-null     object
 4

In [30]:
employee_df.to_csv('Staging_Layer/employee.csv', index=False)
course_df.to_csv('Staging_Layer/course.csv', index=False)
employee_course_df.to_csv('Staging_Layer/employee_course.csv',index=False)

Mart Layer